In [1]:
import os
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant


In [2]:
from pdf2image import convert_from_path
import nltk
import os

poppler_path = r"C:\poppler\poppler-24.02.0\Library\bin"  # Replace with your actual Poppler bin path
os.environ["PATH"] += os.pathsep + poppler_path
# Set the path to Tesseract executable
tesseract_path = r'C:\Tesseract-OCR'

# Add Tesseract path to the PATH environment variable
os.environ['PATH'] += os.pathsep + tesseract_path

# # Set NLTK data path
# nltk_data_path = r'C:\Users\Govind\AppData\Roaming\nltk_data'
# nltk.data.path.append(nltk_data_path)




In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Govind/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Govind/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import nltk
import os

# Set a custom directory for NLTK data
nltk_data_dir = os.path.expanduser('~/nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

# Download NLTK data to the custom directory
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('averaged_perceptron_tagger', download_dir=nltk_data_dir)

[nltk_data] Downloading package punkt to C:\Users\Govind/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Govind/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# Load the embedding model
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

# Load PDF documents
loader = DirectoryLoader('data/', glob="**/*.pdf", show_progress=True, loader_cls=UnstructuredFileLoader)
documents = loader.load()


e:\medico\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3/3 [00:57<00:00, 19.22s/it]


In [6]:

# Assuming documents and embeddings are already defined
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=70)
texts = text_splitter.split_documents(documents)

url = "http://localhost:6333"
qdrant = Qdrant.from_documents(
    texts, 
    embeddings, 
    url=url,
    prefer_grpc=False,
    collection_name="vector_db"
)

print("Vector DB Successfully Created!")

Vector DB Successfully Created!


In [2]:

# # Split text into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=70)
# texts = text_splitter.split_documents(documents)

# # Create the Vector Database
# url = "http://localhost:6333"
# qdrant = Qdrant.from_documents(
#     texts, 
#     embeddings, 
#     url=url,
#     prefer_grpc=False,
#     collection_name="vector_db"
# )

# print("Vector DB Successfully Created!")


In [1]:
pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.82.tar.gz (50.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Failed to build llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.9.8 using CMake 3.30.0 (wheel)
      *** Configuring CMake...
      2024-07-20 00:01:31,489 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\Govind\AppData\Local\Temp\tmp1i9pw1g3\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error o

In [ ]:
from langchain import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import SystemMessagePromptTemplate
from fastapi import FastAPI, Request, Form, Response
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.encoders import jsonable_encoder
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
import os
import json
import gradio as gr

# Load a local BioMistral LLM model with specified settings
local_llm = "BioMistral-7B.Q4_K_M.gguf"
llm = LlamaCpp(model_path=local_llm, temperature=0.3, max_tokens=2048, top_p=1, n_ctx=2048)

# Define a prompt template to guide LLM responses
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Chat History: {chat_history}
Question: {question}

Only return the helpful answer. Answer must be detailed and well explained.
Helpful answer:
"""

# Create sentence embeddings model for representing text as vectors
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

# Connect to a Qdrant vector database for storing and retrieving information
url = "http://localhost:6333"
client = QdrantClient(url=url, prefer_grpc=False)
db = Qdrant(client=client, embeddings=embeddings, collection_name="vector_db")

# Initialize a retriever for searching within the database
retriever = db.as_retriever(search_kwargs={"k":1})

# Create an empty chat history list to track conversation
chat_history = []

# Create a ConversationalRetrievalChain, combining LLM and retriever for context-aware answers
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever)

# Define the actual prediction function
def predict(message, history):
    # Format history for compatibility with LangChain
    history_langchain_format = []

    # Guide LLM with prompt and conversation history to generate a response
    prompt = PromptTemplate(template=prompt_template, input_variables=["chat_history", 'message'])
    response = chain({"question": message, "chat_history": chat_history})

    # Update chat history with the current turn (message and answer)
    answer = response['answer']
    chat_history.append((message, answer))

    # Format history for Gradio interface display
    history_langchain_format = []
    for input_question, bot_answer in history:
        history_langchain_format.append([input_question, bot_answer])

    history_langchain_format.append([message, answer])

    return history_langchain_format, answer

# Create a chat interface using Gradio
gr.ChatInterface(predict).launch()
